In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../Datasets/spam.csv')
df.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [3]:
df.groupby("Category").describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
df['spam'] = df["Category"].apply(lambda x: 1 if x == 'spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.Message,df.spam,test_size=0.25)

In [7]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import sklearn
print(sklearn.__version__)

1.5.0


In [9]:
import sys
print(sys.executable)


/usr/bin/python3


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [12]:
model_params = {
    'svm':{
        'model' : svm.SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10],
        }

    },
    'naive_bayes_gaussian':{
        'model':GaussianNB(),
        'params':{}
    },
    'naive_bayes_multinomial':{
        'model':MultinomialNB(),
        'params':{}
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    },
    'decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy']
        }
    }

}

In [17]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

X_train_count_dense = X_train_count.toarray()

import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf = RandomizedSearchCV(mp['model'], mp['params'], 
                             n_iter=10, cv=5, 
                             return_train_score=False, 
                             n_jobs=-1, 
                             random_state=42)
    clf.fit(X_train_count_dense,y_train)hyper parameter t
    scores.append({-
        'model' : model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

/home/g0pz/.local/lib/python3.11/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KeyboardInterrupt: 

In [26]:
from sklearn.naive_bayes import MultinomialNB
model = LogisticRegression(C=10)
model.fit(X_train_count,y_train)

LogisticRegression(C=10)

In [27]:
emails = [
    'Hey mohan, can we get together to watch football game tomorrow?' ,
    'upto discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 0])

In [28]:
X_test_count = v.transform(X_test)
model.score(X_test_count,y_test)

0.9877961234745154

## pipeline method will be better option

In [29]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [30]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [31]:
clf.predict(emails)

array([0, 1])

In [32]:
clf.score(X_test,y_test)

0.9856424982053122